### Getting the list of Artists and Tracks

In [1]:
import pandas as pd
import pymongo
import getlyrics

In [2]:
all_files = pd.read_csv("Classification/all_files.csv")

In [3]:
artist_track = all_files[["Artist", "Track Name"]]

In [4]:
artist_tracks_unique = artist_track.drop_duplicates()

In [5]:
artist_tracks_unique = artist_tracks_unique.dropna()
artist_tracks_unique = artist_tracks_unique.reset_index()
artist_tracks_unique.drop(["index"], inplace=True, axis=1)

In [6]:
artist_tracks_unique.shape

(3770, 2)

In [7]:
artist_tracks_unique = artist_tracks_unique.head(5)

In [8]:
artist_tracks_unique_dict_ls = artist_tracks_unique.to_dict('records')

In [9]:
len(artist_tracks_unique_dict_ls)

5

In [17]:
not_found_lyrics = []
all_found_lyrics = []
for artist_track in artist_tracks_unique_dict_ls:
    artist_track_lyrics = {}
    try:
        lyrics = getlyrics.get_lyrics(artist_track["Artist"],artist_track["Track Name"])

        artist_track_lyrics["_id"] = artist_track["Artist"] + "_" + artist_track["Track Name"].replace(" ", "_")
        artist_track_lyrics["Lyrics"] = lyrics
        all_found_lyrics.append(artist_track_lyrics)            
    except:
        not_found_lyrics.append(artist_track)

In [18]:
perc_found = (len(artist_tracks_unique_dict_ls) / len(all_found_lyrics)) * 100
print(len(all_found_lyrics), len(not_found_lyrics), (perc_found))

5 0 100.0


In [20]:
len(not_found_lyrics)

0

### Clean the retrived data

In [13]:
all_available_lyrics = pd.DataFrame(all_found_lyrics)

In [14]:
all_available_lyrics.shape

(5, 4)

In [15]:
all_available_lyrics

,Artist,Track Name,_id,Lyrics
0,*NSYNC,"Merry Christmas, Happy Holidays","*NSYNC_Merry_Christmas,_Happy_Holidays",Exception occurred \nlist index out of range
1,187 Strassenbande,Extasy,187 Strassenbande_Extasy,Exception occurred \nlist index out of range
2,187 Strassenbande,Millionr,187 Strassenbande_Millionr,Exception occurred \nlist index out of range
3,2 Chainz,4 AM,2 Chainz_4_AM,Exception occurred \nlist index out of range
4,2 Chainz,Bigger Than You (feat. Drake & Quavo),2 Chainz_Bigger_Than_You_(feat._Drake_&_Quavo),Exception occurred \nlist index out of range


In [16]:
all_available_lyrics = pd.DataFrame(all_available_lyrics)
all_available_lyrics["ModifiedLyrics"] = all_available_lyrics["Lyrics"].apply( lambda x : "" if("Error 404: ") in x else x )
all_available_lyrics["ModifiedLyrics"] = all_available_lyrics["Lyrics"].apply( lambda x : "" if("Exception occurred") in x else x )
len_available_lyrics = len(all_available_lyrics["ModifiedLyrics"].unique().tolist())
all_available_lyrics_available = all_available_lyrics[all_available_lyrics['track details'].notna()]

the_keys = list(all_available_lyrics_available[["track details"]].iloc[0].tolist()[0].keys())

for a_key in the_keys:
    all_available_lyrics_available[a_key] = all_available_lyrics_available["track details"].apply( lambda x : x[a_key] ) 
    
all_available_lyrics_available.drop(["track details", "Lyrics"], axis=1, inplace=True)

all_available_lyrics_available.to_csv("AllAvailableLyricsNew.csv", index=False)

all_available_lyrics_available_dict_ls = all_available_lyrics_available.to_dict('records')

KeyError: 'track details'

### Write the read items to the database

In [ ]:
if( len(all_available_lyrics_available_dict_ls) > 0):
    myclient = pymongo.MongoClient("mongodb://localhost:27017/")
    mydb = myclient["spotifytop200global"]
    mycol = mydb["lyricsfinal"]

    x = mycol.insert_many(all_available_lyrics_available_dict_ls)